In [1]:
import argparse
import itertools
import json
import math
import os
import sys
import timeit

import dolfinx
import gmsh
import h5py
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pyvista
import pyvista as pv
import pyvistaqt as pvqt
import subprocess
import ufl
import warnings

from basix.ufl import element, mixed_element
from dolfinx import cpp, default_real_type, default_scalar_type, fem, io, la, mesh, nls, plot
from dolfinx.fem import petsc
from dolfinx.io import gmshio, VTXWriter, XDMFFile
from dolfinx.nls import petsc as petsc_nls
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells
from IPython.display import Image

from mpi4py import MPI
from petsc4py import PETSc
from ufl import (Circumradius, FacetNormal, SpatialCoordinate, TrialFunction, TestFunction,
                 dot, div, dx, ds, dS, grad, inner, grad, avg, jump)

import commons, configs, geometry, utils

warnings.simplefilter("ignore")

In [2]:
def arctanh(y):
    return 0.5 * ufl.ln((1 + y) / (1 - y))

def ocv(c, cmax=35000):
    xi = c / cmax
    return 3.25 - 0.5 * arctanh(xi)

In [3]:
# c_vals = np.linspace(35000, -35000)
# ocvs = ocv(c_vals)
# fig, ax = plt.subplots()
# ax.plot(c_vals, ocvs)
# plt.tight_layout()

In [4]:
comm = MPI.COMM_WORLD
encoding = io.XDMFFile.Encoding.HDF5
micron = 1e-6
dimensions = "150-40-0"
mesh_folder = "output/ssb_unstructured/150-40-0/20-55-20/5.0e-06/"
LX, LY, LZ = [float(vv) * micron for vv in dimensions.split("-")]
workdir = os.path.join(mesh_folder, "tertiary")
utils.make_dir_if_missing(workdir)
output_meshfile = os.path.join(mesh_folder, 'mesh.msh')
lines_h5file = os.path.join(mesh_folder, 'lines.h5')
potential_resultsfile = os.path.join(workdir, "potential.bp")
concentration_resultsfile = os.path.join(workdir, "concentration.bp")
concentration_xdmf_file = os.path.join(workdir, "concentration.xdmf")
# current_dist_file = os.path.join(workdir, f"current-y-positions-{str(args.Wa_p)}-{str(args.kr)}.png")
# reaction_dist_file = os.path.join(workdir, f"reaction-dist-{str(args.Wa_p)}-{str(args.kr)}.png")
current_resultsfile = os.path.join(workdir, "current.bp")
simulation_metafile = os.path.join(workdir, "simulation.json")

markers = commons.Markers()

# ### Read input geometry
partitioner = mesh.create_cell_partitioner(mesh.GhostMode.shared_facet)
domain, ct, ft = gmshio.read_from_msh(output_meshfile, comm, partitioner=partitioner)
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(tdim, fdim)
x = SpatialCoordinate(domain)

# tag internal facets as 0
ft_imap = domain.topology.index_map(fdim)
num_facets = ft_imap.size_local + ft_imap.num_ghosts
indices = np.arange(0, num_facets)
values = np.zeros(indices.shape, dtype=np.intc)

values[ft.indices] = ft.values

ft = mesh.meshtags(domain, fdim, indices, values)
ct = mesh.meshtags(domain, tdim, ct.indices, ct.values)

dx = ufl.Measure("dx", domain=domain, subdomain_data=ct, metadata={"quadrature_degree": 4})
ds = ufl.Measure("ds", domain=domain, subdomain_data=ft, metadata={"quadrature_degree": 4})

f_to_c = domain.topology.connectivity(fdim, tdim)
c_to_f = domain.topology.connectivity(tdim, fdim)
charge_xfer_facets = ft.find(markers.electrolyte_v_positive_am)

other_internal_facets = ft.find(0)
int_facet_domain = []
for f in charge_xfer_facets:
    if f >= ft_imap.size_local or len(f_to_c.links(f)) != 2:
        continue
    c_0, c_1 = f_to_c.links(f)[0], f_to_c.links(f)[1]
    subdomain_0, subdomain_1 = ct.values[[c_0, c_1]]
    local_f_0 = np.where(c_to_f.links(c_0) == f)[0][0]
    local_f_1 = np.where(c_to_f.links(c_1) == f)[0][0]
    if subdomain_0 > subdomain_1:
        int_facet_domain.append(c_0)
        int_facet_domain.append(local_f_0)
        int_facet_domain.append(c_1)
        int_facet_domain.append(local_f_1)
    else:
        int_facet_domain.append(c_1)
        int_facet_domain.append(local_f_1)
        int_facet_domain.append(c_0)
        int_facet_domain.append(local_f_0)

other_internal_facet_domains = []
for f in other_internal_facets:
    if f >= ft_imap.size_local or len(f_to_c.links(f)) != 2:
        continue
    c_0, c_1 = f_to_c.links(f)[0], f_to_c.links(f)[1]
    subdomain_0, subdomain_1 = ct.values[[c_0, c_1]]
    local_f_0 = np.where(c_to_f.links(c_0) == f)[0][0]
    local_f_1 = np.where(c_to_f.links(c_1) == f)[0][0]
    other_internal_facet_domains.append(c_0)
    other_internal_facet_domains.append(local_f_0)
    other_internal_facet_domains.append(c_1)
    other_internal_facet_domains.append(local_f_1)
int_facet_domains = [(markers.electrolyte_v_positive_am, int_facet_domain)]#, (0, other_internal_facet_domains)]

dS = ufl.Measure("dS", domain=domain, subdomain_data=int_facet_domains)

Info    : Reading 'output/ssb_unstructured/150-40-0/20-55-20/5.0e-06/mesh.msh'...
Info    : 55 entities
Info    : 5106 nodes
Info    : 10380 elements
Info    : Done reading 'output/ssb_unstructured/150-40-0/20-55-20/5.0e-06/mesh.msh'


In [5]:
# ### Function Spaces
P1 = element("DG", domain.basix_cell(), 1, dtype=default_real_type)
P2 = element("DG", domain.basix_cell(), 1, dtype=default_real_type)
# V = fem.functionspace(domain, mixed_element([P1, P2]))
V = fem.functionspace(domain, ("DG", 1, (2,)))
W = fem.functionspace(domain, ("DG", 1, (3,)))
Z = fem.functionspace(domain, ("CG", 1, (3,)))
Q = fem.functionspace(domain, ("DG", 0))
u = fem.Function(V, name='potential')

φ, c = ufl.split(u)

u0 = fem.Function(V, name='potential')


# v = ufl.TestFunction(V)
δφ, δc = ufl.TestFunctions(V)

current_h = fem.Function(W, name='current_density')
kappa = fem.Function(Q, name='conductivity')
D = fem.Function(Q, name='diffusivity')
n = ufl.FacetNormal(domain)
x = ufl.SpatialCoordinate(domain)
h = ufl.CellDiameter(domain)
h_avg = avg(h)

In [6]:
cells_elec = ct.find(markers.electrolyte)
kappa_elec = 0.1
kr = 1
kappa_pos_am = kappa_elec/kr
cells_pos_am = ct.find(markers.positive_am)

kappa.x.array[cells_elec] = np.full_like(cells_elec, kappa_elec, dtype=default_scalar_type)
kappa.x.array[cells_pos_am] = np.full_like(cells_pos_am, kappa_pos_am, dtype=default_scalar_type)

D.x.array[cells_pos_am] = np.full_like(cells_pos_am, 1e-15, dtype=default_scalar_type)
D.x.array[cells_elec] = np.full_like(cells_elec, 1e-10, dtype=default_scalar_type)

u0.sub(1).x.array[cells_elec] = np.full_like(cells_elec, 0, dtype=default_scalar_type)
u0.sub(1).x.array[cells_pos_am] = np.full_like(cells_pos_am, 32500, dtype=default_scalar_type)
u0.x.scatter_forward()
φ0, c0 = ufl.split(u0)

In [7]:
dt = 1e-3
TIME = 50 * dt
voltage = 1
kappa_elec = 0.1
kappa_pos_am = 0.2
faraday_const = 96485
R = 8.3145
T = 298
Wa_p = 1e0
Wa_n = 1e-3

#### Piece-wise properties

In [8]:
f = fem.Constant(domain, PETSc.ScalarType(0))
g = fem.Constant(domain, PETSc.ScalarType(0))

u_left = fem.Function(V).sub(0)
with u_left.vector.localForm() as u0_loc:
    u0_loc.set(0)
u_right = fem.Function(V).sub(0)
with u_right.vector.localForm() as u1_loc:
    u1_loc.set(voltage)

i0_n = kappa_elec * R * T / (Wa_n * faraday_const * LX)
i0_p = kappa_elec * R * T / (Wa_p * faraday_const * LX)

u_ocv = ocv(c("+"))
V_left = 0

alpha = 100#args.gamma
gamma = 100#args.gamma
i_loc = -inner((kappa * grad(φ))('+'), n("+"))
u_jump = 2 * ufl.ln(0.5 * i_loc/i0_p + ufl.sqrt((0.5 * i_loc/i0_p)**2 + 1)) * (R * T / faraday_const)

Fφ = kappa * inner(grad(φ), grad(δφ)) * dx - f * δφ * dx - kappa * inner(grad(φ), n) * δφ * ds

# Add DG/IP terms
Fφ += - avg(kappa) * inner(jump(φ, n), avg(grad(δφ))) * dS#(0)
# F += - inner(utils.jump(kappa * u, n), avg(grad(v))) * dS(0)
Fφ += - inner(avg(kappa * grad(φ)), jump(δφ, n)) * dS#(0)
# F += + avg(u) * inner(utils.jump(kappa, n), avg(grad(v))) * dS(0)
Fφ += alpha / h_avg * avg(kappa) * inner(jump(δφ, n), jump(φ, n)) * dS#(0)

# Internal boundary
Fφ += + avg(kappa) * dot(avg(grad(δφ)), (u_jump + u_ocv) * n('+')) * dS(markers.electrolyte_v_positive_am)
Fφ += -alpha / h_avg * avg(kappa) * dot(jump(δφ, n), (u_jump + u_ocv) * n('+')) * dS(markers.electrolyte_v_positive_am)

# # Symmetry
Fφ += - avg(kappa) * inner(jump(φ, n), avg(grad(δφ))) * dS(markers.electrolyte_v_positive_am)

# # Coercivity
Fφ += alpha / h_avg * avg(kappa) * inner(jump(φ, n), jump(δφ, n)) * dS(markers.electrolyte_v_positive_am)

# Nitsche Dirichlet BC terms on left and right boundaries
Fφ += - kappa * (φ - u_left) * inner(n, grad(δφ)) * ds(markers.left)
Fφ += -gamma / h * (φ - u_left) * δφ * ds(markers.left)
Fφ += - kappa * (φ - u_right) * inner(n, grad(δφ)) * ds(markers.right) 
Fφ += -gamma / h * (φ - u_right) * δφ * ds(markers.right)

# Nitsche Neumann BC terms on insulated boundary
Fφ += -g * δφ * ds(markers.insulated_electrolyte) + gamma * h * g * inner(grad(δφ), n) * ds(markers.insulated_electrolyte)
Fφ += - gamma * h * inner(inner(grad(φ), n), inner(grad(δφ), n)) * ds(markers.insulated_electrolyte)
Fφ += -g * δφ * ds(markers.insulated_positive_am) + gamma * h * g * inner(grad(δφ), n) * ds(markers.insulated_positive_am)
Fφ += - gamma * h * inner(inner(grad(φ), n), inner(grad(δφ), n)) * ds(markers.insulated_positive_am)

# kinetics boundary - neumann
# Fφ += - gamma * h * inner(inner(kappa * grad(u), n), inner(grad(v), n)) * ds(markers.left)
# Fφ -= - gamma * h * 2 * i0_n * ufl.sinh(0.5 * faraday_const / R / T * (V_left - u - 0)) * inner(grad(v), n) * ds(markers.left)

In [9]:
Fc = c * δc * dx - c0 * δc * dx

Fct = D * inner(grad(c), grad(δc)) * dx - f * δc * dx - D * inner(grad(c), n) * δc * ds

# Add DG/IP terms
Fct += - avg(D) * inner(jump(c, n), avg(grad(δc))) * dS#(0)
Fct += - inner(avg(D * grad(c)), jump(δc, n)) * dS#(0)
Fct += alpha / h_avg * avg(D) * inner(jump(δc, n), jump(c, n)) * dS#(0)

# zero-concentration
# Fct += - kappa * (c - 0) * inner(n, grad(δc)) * ds(markers.left)
# Fct += -gamma / h * (c - 0) * δc * ds(markers.left)

# insulated
# Fct += g * δc * ds(markers.insulated_electrolyte)
# Fct += g * δc * ds(markers.insulated_positive_am)
# Fct += g * δc * ds(markers.right)
# Fct += 1e-3 * δc * ds(markers.left)
Fct += -g * δc * ds(markers.insulated_electrolyte) + gamma * h * g * inner(grad(δc), n) * ds(markers.insulated_electrolyte)
Fct += - gamma * h * inner(inner(grad(c), n), inner(grad(δc), n)) * ds(markers.insulated_electrolyte)
Fct += -g * δc * ds(markers.insulated_positive_am) + gamma * h * g * inner(grad(δc), n) * ds(markers.insulated_positive_am)
Fct += - gamma * h * inner(inner(grad(c), n), inner(grad(δc), n)) * ds(markers.insulated_positive_am)
Fct += -g * δc * ds(markers.insulated_positive_am) + gamma * h * g * inner(grad(δc), n) * ds(markers.right)
Fct += - gamma * h * inner(inner(grad(c), n), inner(grad(δc), n)) * ds(markers.right)

# Internal boundary
Fct += + (1/faraday_const) * inner(jump(δc, n), avg(kappa * grad(φ))) * dS(markers.electrolyte_v_positive_am)
# Fct += -alpha / h_avg * avg(D) * dot(jump(δc, n), (u_jump + u_ocv) * n('+')) * dS(markers.electrolyte_v_positive_am)

# # # Symmetry
Fct += - avg(D) * inner(jump(c, n), avg(grad(δc))) * dS(markers.electrolyte_v_positive_am)

# # # Coercivity
Fct += alpha / h_avg * avg(D) * inner(jump(c, n), jump(δc, n)) * dS(markers.electrolyte_v_positive_am)

Fc += dt * Fct

F = Fφ + Fc

In [10]:
problem = petsc.NonlinearProblem(F, u)
solver = petsc_nls.NewtonSolver(comm, problem)
solver.convergence_criterion = "incremental"
# solver.maximum_iterations = 100
# solver.rtol = args.rtol
# solver.atol = args.atol

ksp = solver.krylov_solver

opts = PETSc.Options()
ksp.setMonitor(lambda _, it, residual: print(it, residual))
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"

# For factorisation prefer superlu_dist, then MUMPS, then default
# if sys.hasExternalPackage("superlu_dist"):
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "superlu_dist"
# elif sys.hasExternalPackage("mumps"):
#     opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"

ksp.setFromOptions()
t = 0.0
c_vtx = VTXWriter(comm, concentration_resultsfile, u, engine="BP5")
c_vtx.write(0.0)
# file = XDMFFile(comm, concentration_xdmf_file, "w", encoding=XDMFFile.Encoding.ASCII)
# file.write_mesh(domain)
# file.write_function(c0, t)

while t < TIME:
    t += dt
    print(f"Time: {t}")
    n_iters, converged = solver.solve(u)
    u.x.scatter_forward()
    # φ0, c0 = ufl.split(u)
    u0.x.array[:] = u.x.array
    c_vtx.write(t)
    # file.write_function(c, t)
# file.close()
c_vtx.close()

Time: 0.001


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

0 599.7025886926848
1 1.3739548529156055e-10


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

0 10.337536243317942
1 1.513795750189533e-13


...

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

0 2.4481140791934787e-10
1 1.4821481027861177e-13
0 0.0001275034904726561
1 4.781709863225223e-18


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

0 1.2087763618199228e-09
1 1.6511959531710243e-21
0 2.184413499498463e-12
1 8.634234220477836e-24


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec